In [1]:
# !pip install langchain
# !pip install chromadb
# !pip install transformers
# !pip install sentence_transformers

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader, DirectoryLoader
from chromadb.utils import embedding_functions

In [3]:
#loader = TextLoader('../RadarPlainText/Radar Basics1.txt')
loader = DirectoryLoader('../ThreatInfo/', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()
display(len(documents))

1

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [20]:

# Save embedding to disk and load if available
import torch
import os
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Downloading from HF took forever, check if embedding is on disk and use that
if os.path.exists("instructor-xl.pt"):
    # Load embedding from disk
    embedding = torch.load("instructor-xl.pt")
    print("Loaded embeddings from disk.")
else:
    # If the embeddings are not found on disk, download them from Hugging Face
    print("Embeddings not found on disk. Downloading from Hugging Face...")
    embedding = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", model_kwargs={"device": "cuda"})
    # Save to disk
#     torch.save(embedding,"instructor-xl.pt")
#     print("Saved embeddings to disk.")
    
    
#embedding = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
#                                                       model_kwargs={"device": "cuda"})

Embeddings not found on disk. Downloading from Hugging Face...
load INSTRUCTOR_Transformer
max_seq_length  512


In [21]:
vectordb = Chroma.from_documents(texts, embedding)

In [22]:
# Retriever
#retriever = vectordb.as_retriever()
# Return the top k documents
retriever = vectordb.as_retriever(search_kwargs={"k": 8})


In [23]:
# Test retriever
# docs = retriever.get_relevant_documents("What is a radar waveform?")
# display(len(docs))
# display(docs[0])
# display(docs[2])
# display(retriever.search_type)
# display(retriever.search_kwargs)


In [78]:
# Use Opal Server

from _OpalLLM import OpalLLM

if 1:
    #model_name="lmsys/vicuna-33b"
    #model_name="databricks/dolly-v2-12b"
    model_name="meta-llama/Llama-2-13b-chat-hf"
#     model_name = "tiiuae/falcon-40b-instruct"
    #model_name="CarperAI/stable-vicuna-13b-delta"
    
    local_llm=OpalLLM(model=model_name,
                      temperature=0.5,
                      top_k=60,
                      top_p=0.95,
                      max_tokens=200,
                      repetition_penalty=1.15)
    display(local_llm.model)
    
    

'meta-llama/Llama-2-13b-chat-hf'

In [79]:
# RetrievalQA Chain
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm=local_llm,
                                chain_type="stuff",
                                retriever=retriever,
                                return_source_documents=True,
                                verbose=False)


In [80]:
# Format this text
import textwrap

def trim_string(input_string):
    input_string = str(input_string)
    trim_index = input_string.find("### Human:")
    if trim_index != -1:  # If the phrase is found
        return input_string[:trim_index]
    else:
        return input_string  # If the phrase isn't found, return the original string
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    temp_resp = wrap_text_preserve_newlines(llm_response['result'])
    temp_resp = trim_string(temp_resp)
    if "</s>" in temp_resp:
        ind = temp_resp.find("</s>")
        temp_resp = temp_resp[:ind]
    print(temp_resp)
    print('\n\nSources:')
    clean_source_ls = []
    for source in llm_response["source_documents"]:
        if source.metadata['source'] not in clean_source_ls:
            clean_source_ls.append(source.metadata['source'])
    for sources in clean_source_ls:
        print(sources)

In [92]:
# Formatted Test
# query = "What is Range Doppler Processing used for? Give examples of the applications and explain how doppler is used from a physics perspective"
query = "Summarize and list the main threats from China that I should know about as a radar specialist."
llm_response = qa_chain(query)
process_llm_response(llm_response)

  Based on the text provided, here are some key threats from China that you may want to be aware of as a radar
specialist:

* China has launched the most satellites of any nation last year, indicating their intent to project hard and
soft power through on-orbit military support capabilities.
* China has demonstrated counterspace demonstrations such as hypersonic missile launches and co-orbital
rendezvous with other satellites.
* China has a growing suite of jamming and spoofing electronic warfare capabilities to be used against space
and non-space signals alike.
* Little is known about China's cyber counterspace capabilities, but their cyber capabilities in other domains
suggest a strong foundation for potential cyber counterspace capabilities.


Sources:
../ThreatInfo/220404_Harrison_SpaceThreatAssessment2022.txt
